# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 

### Associated lectures: [Lectures 17 and 18](https://github.com/UBC-CS/cpsc330-2024s/tree/main/lectures)

**See PrairieLearn for _due date_ and _submission_**

## Imports

In [1]:
import os
from hashlib import sha1

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>

_points: 2_

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size $\leq$ 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Be sure to follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024s/blob/main/docs/homework_instructions.md).
- Upload the .ipynb file to PrairieLearn.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 17, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [2]:
pip install scipy==1.10.1

Note: you may need to restart the kernel to use updated packages.


In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings

_Points:_ 2

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

In [6]:
words = ["car","king", "computer", "music"] # My 4 words

In [7]:
# Get the top 5 similar words for every word in my words list
for word in words:
    similars = glove_wiki_vectors.most_similar(word, topn=5) # Get the top 5 similar words to word
    print(f"Words similar to '{word}':");
    for similar in similars: # Print the Results and their score
        print(f" {similar}")

Words similar to 'car':
 ('vehicle', 0.8630837798118591)
 ('truck', 0.8597878813743591)
 ('cars', 0.837166965007782)
 ('driver', 0.8185911178588867)
 ('driving', 0.781263530254364)
Words similar to 'king':
 ('prince', 0.7682328820228577)
 ('queen', 0.7507690787315369)
 ('son', 0.7020888328552246)
 ('brother', 0.6985775232315063)
 ('monarch', 0.6977890729904175)
Words similar to 'computer':
 ('computers', 0.8751984238624573)
 ('software', 0.8373122215270996)
 ('technology', 0.7642159461975098)
 ('pc', 0.7366448640823364)
 ('hardware', 0.7290390729904175)
Words similar to 'music':
 ('musical', 0.8128045797348022)
 ('songs', 0.7978180646896362)
 ('dance', 0.7896507382392883)
 ('pop', 0.7862942218780518)
 ('recording', 0.7650765776634216)


<!-- END QUESTION -->

<br><br>

### 1.2 Word similarity using pre-trained embeddings

_Points:_ 2

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [8]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

In [9]:
coast_shore_similarity = None
clothes_closet_similarity = None
old_new_similarity = None
smart_intelligent_similarity = None
dog_cat_similarity = None
tree_lawyer_similarity = None


# Printing the results
for word1, word2 in word_pairs:
    similarity = glove_wiki_vectors.similarity(word1, word2)
    print(f"'{word1}' and '{word2}': {similarity:}")


'coast' and 'shore': 0.7000271677970886
'clothes' and 'closet': 0.5462760329246521
'old' and 'new': 0.6432487964630127
'smart' and 'intelligent': 0.7552732825279236
'dog' and 'cat': 0.8798074722290039
'tree' and 'lawyer': 0.07671945542097092


In [10]:
coast_shore_similarity = 0.7000271677970886
clothes_closet_similarity = 0.5462760329246521
old_new_similarity = 0.6432487964630127
smart_intelligent_similarity = 0.7552732825279236
dog_cat_similarity = 0.8798074722290039
tree_lawyer_similarity = 0.07671945542097092

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings

_Points:_ 6

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.

In [11]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [12]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [13]:
glove_wiki_vectors.similarity("white", "rich")

0.44723597

In [14]:
glove_wiki_vectors.similarity("black", "rich")

0.51745194

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Type your answer here, replacing this text._

In [15]:
analogy("nurse","doctor", "scientist")

nurse : doctor :: scientist : ?


,Analogy word,Score
0,physicist,0.695650
1,science,0.649368
2,author,0.648932
3,professor,0.646399
4,researcher,0.625816
5,expert,0.622294
6,scientists,0.617924
7,dr.,0.600684
8,astronomer,0.586077
9,mathematician,0.585461


**Results show different fields across science or professional fields, meaning there is less bias since there are no stereotypes and only professions**

In [16]:
analogy("man","strong","woman")

man : strong :: woman : ?


,Analogy word,Score
0,stronger,0.726966
1,weak,0.657428
2,robust,0.644944
3,strongest,0.634013
4,despite,0.631509
5,support,0.627726
6,growing,0.625404
7,concern,0.605287
8,particularly,0.603681
9,reflected,0.600039


**This one has bias as it appears the models second highest score is 'weak'**

In [17]:
analogy("man", "engineer", "woman")

man : engineer :: woman : ?


,Analogy word,Score
0,technician,0.662020
1,educator,0.611515
2,engineers,0.592235
3,surgeon,0.588020
4,contractor,0.587330
5,engineering,0.568600
6,nurse,0.556342
7,chemist,0.554719
8,pioneer,0.546008
9,physician,0.545960


**This one also has bias as it appears some of the top results include educator and nurse which is associating women with tradional female-dominated roles**

In [18]:
analogy("rich", "successful", "poor")

rich : successful :: poor : ?


,Analogy word,Score
0,success,0.637987
1,failure,0.617793
2,failed,0.601898
3,slow,0.594060
4,performance,0.593510
5,first,0.590567
6,continued,0.589850
7,result,0.581473
8,failing,0.581092
9,difficult,0.578083


**This also has slight bias as the top options include failure, failed, slow which associates poverty with failure**

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion

_Points:_ 4

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

1. The models appear to be trained quite well, and capture a lot of the relationships. However, as we saw above they do have bias when it comes to gender and economical bias.
2. A model that reinforces gender stereotypes like in the engineering example, could cause harm when hiring engineers as it will dismiss women for men for these positions.

<!-- END QUESTION -->

<br><br>

### 1.5 Classification with pre-trained embeddings 

_Points:_ 8

In lecture 17, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under the data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`, be sure to filter out stop words), calculate the train and test accuracy and show a [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.
2. Train logistic regression with average embedding representation extracted using spaCy, calculate the train and test accuracy and show the [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.

In [19]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [20]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [21]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [22]:
import spacy
import en_core_web_md

# nlp = spacy.load("en_core_web_md")
nlp = en_core_web_md.load()

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Type your answer here, replacing this text._

In [23]:
from sklearn.metrics import classification_report
bow_train_accuracy = None
bow_test_accuracy = None
bow_report = None # use digits=2

# Create bag of words
vectorizer = CountVectorizer(stop_words='english')
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Train logistic regression
bow_model = LogisticRegression(max_iter=1000)
bow_model.fit(X_train_bow, y_train)

# Evaluating it
y_train_pred_bow = bow_model.predict(X_train_bow)
y_test_pred_bow = bow_model.predict(X_test_bow)

# Show classification report
bow_train_accuracy = bow_model.score(X_train_bow, y_train)
bow_test_accuracy = bow_model.score(X_test_bow, y_test)
bow_report = classification_report(y_test, y_test_pred_bow,digits=2) # use digits=2
print("Train Accuracy:", bow_train_accuracy)
print("Test Accuracy:", bow_test_accuracy)
print(bow_report)

Train Accuracy: 0.9558005461717407
Test Accuracy: 0.8178386031146767
                  precision    recall  f1-score   support

     achievement       0.79      0.87      0.83      1302
       affection       0.90      0.91      0.91      1423
         bonding       0.91      0.85      0.88       492
enjoy_the_moment       0.60      0.55      0.57       469
        exercise       0.91      0.57      0.70        74
         leisure       0.73      0.70      0.72       407
          nature       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.80      0.70      0.74      4238
    weighted avg       0.82      0.82      0.82      4238



In [24]:
X_train_embeddings = None
X_test_embeddings = None

# average for X_train
X_train_embed = X_train.apply(lambda text: nlp(text).vector)

# average for X_test
X_test_embed = X_test.apply(lambda text: nlp(text).vector)

In [25]:
X_train_embed

hmid
50152     [-0.6908737, -0.7552159, -3.2796504, -1.111955...
97451     [0.021659326, -1.051312, -4.0633607, -1.818071...
77308     [-0.10163904, -1.6745467, -1.3914334, -3.23056...
41407     [-1.5028296, -0.24550462, -2.919757, -1.312146...
74665     [-0.5038384, -0.45007557, -4.626111, -2.820388...
                                ...                        
66548     [-0.24387683, 0.1008765, -5.652421, -2.35868, ...
115391    [-1.73938, -0.46098447, -2.6973324, -2.1435125...
39029     [-1.0594908, 2.290304, -3.9372292, -1.1838707,...
111033    [-2.2703135, -2.76444, -2.6081924, -2.6569943,...
54752     [-2.3019068, 0.2866033, -1.9906557, -0.2490994...
Name: moment, Length: 9887, dtype: object

In [26]:
X_test_embed

hmid
62789     [-0.9584156, 3.125293, -4.034988, -2.0419073, ...
81694     [-1.2998937, 0.6303664, -2.3180046, -2.5071044...
83106     [1.671838, -3.7535603, -2.243932, -6.0349007, ...
112096    [3.2853158, -1.347252, -5.52016, -1.7933264, -...
75708     [-3.2961729, 1.5183955, -3.7267184, -1.1181253...
                                ...                        
127944    [1.5578985, 3.393842, -4.641134, -2.0906985, -...
84472     [0.89856815, 0.06906378, -4.769829, -1.4402249...
123564    [-1.1052524, 0.13804013, -3.5179975, -1.480571...
76296     [0.4270342, 0.9249369, -3.550213, -3.1721811, ...
111881    [-0.97893906, 1.1322981, -1.8153081, -1.661736...
Name: moment, Length: 4238, dtype: object

In [27]:
emb_train_accuracy = None
emb_test_accuracy = None
emb_report = None # use digits=2

embed_model = LogisticRegression(max_iter=4000)
embed_model.fit(list(X_train_embed), y_train)

y_train_pred_embed = embed_model.predict(list(X_train_embed))
y_test_pred_embed = embed_model.predict(list(X_test_embed))

emb_train_accuracy = embed_model.score(list(X_train_embed), y_train)
emb_test_accuracy = embed_model.score(list(X_test_embed), y_test)
emb_report = classification_report(y_test, y_test_pred_embed, digits=2)

print("Train Accuracy:", emb_train_accuracy)
print("Test Accuracy:", emb_test_accuracy)
print(emb_report)

Train Accuracy: 0.8501062000606857
Test Accuracy: 0.7940066068900424
                  precision    recall  f1-score   support

     achievement       0.81      0.83      0.82      1302
       affection       0.86      0.91      0.89      1423
         bonding       0.82      0.77      0.80       492
enjoy_the_moment       0.56      0.51      0.53       469
        exercise       0.68      0.76      0.72        74
         leisure       0.71      0.65      0.68       407
          nature       0.69      0.72      0.70        71

        accuracy                           0.79      4238
       macro avg       0.74      0.74      0.73      4238
    weighted avg       0.79      0.79      0.79      4238



<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion

_Points:_ 6

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

1. The CountVectorizer approach transforms the text into word count matrix, and does not capture the context behind them, only a bag of jumbled words. Whereas Average Embedding uses pre trained word embeddings to create vectores that capture relationships between words
2. The countVectorizer is much easier to read as it has features that correspond to words (counts). Average embedding is dense vectors
3. We are using transfer learning throuhg using pre trained embeddings, which improves the generalization and score of the model as well as being able to determine context 

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [28]:
from sklearn.datasets import fetch_20newsgroups

In [29]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [30]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)

_Points:_ 8

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

_Type your answer here, replacing this text._

In [31]:
newsgroups_train = fetch_20newsgroups(subset='train', categories=cats, remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', categories=cats, remove=('headers', 'footers', 'quotes'))
df_train = pd.DataFrame({'text': newsgroups_train.data, 'target': newsgroups_train.target})
df_train.head()

,text,target
0,"You know, I was reading 18 U.S.C. 922 and some...",6
1,\n\n\nIt's not a bad question: I don't have an...,1
2,"\nActuallay I don't, but on the other hand I d...",1
3,"The following problem is really bugging me,\na...",2
4,\n\n This is the latest from UPI \n\n For...,7


In [32]:
import re

In [33]:
preprocessed_texts = []
for doc in df_train['text']:
    # Removing urls and emails using re (inspired by https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python)
    text = re.sub(r'http\S+|www\S+|@\S+', '', doc)

    # Process text using spacy
    doc = nlp(doc)
    
    tokens = [
        token.lemma_ for token in doc # lemmatize
        if not token.is_stop and not token.is_punct and token.is_alpha # remove stop words, punctuation, and non-alphabetical tokens
    ]
    
    preprocessed_text = ' '.join(tokens)
    
    preprocessed_texts.append(preprocessed_text)

# Add as column to our df
df_train['text_pp'] = preprocessed_texts

# Saving the dataframe to avoid having to run it every time
df_train.to_csv('./data/preprocessed_newsgroups_train.csv', index=False)

df_train[['text', 'text_pp']].head()

,text,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",know read sence wonder help provide paragraph ...
1,\n\n\nIt's not a bad question: I don't have an...,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",Actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,late UPI Foreign Ministry spokesman Ferhat Ata...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification

_Points:_ 2

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps.

<div class="alert alert-warning">

Solution_2_2
    
</div>

**The steps I took for preprocessing were**:
- Removing emails and URLs: since they're unique and they don't contribute to the content of the text (usually emails are just the email of the author)
- Lemmatization and Tokenization: Reducing the words to their base forms as discussed in the lectures and making all the words in different forms common.
- Remove stop words and punctuation: Just like the bag of words approach, we remove stop words which are redundant and are not really meaningful since they are only for sentence flow
- Then I rejoined the test for the document and added it to a dataframe, then saved it into a csv file for later  .

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation

_Points:_ 4

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Type your answer here, replacing this text._

In [34]:
from sklearn.decomposition import LatentDirichletAllocation
vectorizer = CountVectorizer(stop_words='english', max_features=1000)
X_train_counts = vectorizer.fit_transform(df_train['text_pp'])

In [35]:
# create the model (Code inspired by Lecture 17)

n_topics = 8 # number of topics is 8 so we choose 8
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=0
)

lda.fit(X_train_counts)

LatentDirichletAllocation(n_components=8, random_state=0)

In [36]:
print("lda.components_.shape: {}".format(lda.components_.shape))

lda.components_.shape: (8, 1000)


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association

_Points:_ 5

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

In [37]:
import mglearn

# Inspired by Lecture 17 Toy example

no_top_words = 10
feature_names = vectorizer.get_feature_names_out()
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]

mglearn.tools.print_topics(
    topics=range(8),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=no_top_words,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
game          armenian      new           god           israel        
team          say           university    know          jews          
play          people        list          think         state         
year          armenians     gm            people        law           
player        turkish       april         say           right         
good          kill          york          believe       israeli       
win           come          group         thing         jewish        
season        turkey        information   jesus         arab          
think         woman         city          come          people        
hockey        child         san           like          country       


topic 5       topic 6       topic 7       
--------      --------      --------      
gun           file          window        
people        ent

**Topic Label Suggestions**:
1. Topic 0: Hockey or sports (It seams to be talking about games, players, years, winning and losing, and mentions hockey)
2. Topic 1: Looks like the Armenian genocide (top words include armenians, kill, turkey)
3. Topic 2: news about york univesity (Mentions new (lemmitization of news), university, york (assuming york uni).
4. Topic 3: Religion (mentions believe, jesus, know, and god)
5. Topic 4: Palestine conflict/Middle East politics (mentions israel, state, law, rights, and arabs)
6. Topic 5: Gun control (guns, thinking, people, crime control all imply gun control)
7. Topic 6: Image processing program (implied by jpeg, color, output)
8. Topic 7: Windows os (implied by run , lemmitized windows, work, file)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association

_Points:_ 5

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

In [38]:
document_topics = lda.transform(X_train_counts)
document_topics

array([[3.47892519e-03, 3.47827239e-03, 3.47341495e-03, ...,
        3.34655614e-01, 3.47644609e-03, 3.48142452e-03],
       [2.66388779e-03, 2.66446241e-03, 1.74700792e-01, ...,
        2.71348326e-01, 5.40630893e-01, 2.66399920e-03],
       [3.47481033e-03, 3.47479414e-03, 3.48329813e-03, ...,
        3.47686172e-03, 3.47510773e-03, 2.58642702e-01],
       ...,
       [4.03508971e-03, 4.03583986e-03, 4.04152229e-03, ...,
        8.64699634e-01, 4.03419238e-03, 4.03728284e-03],
       [5.95618161e-03, 5.95660724e-03, 5.95633215e-03, ...,
        5.96002226e-03, 1.55337542e-01, 8.08919946e-01],
       [6.35148345e-04, 7.04334052e-01, 6.35765497e-04, ...,
        2.27382517e-01, 1.30983727e-02, 6.35599634e-04]])

In [39]:
# Getting first 5
documents = []
for i in range(0,5):
    documents.append(document_topics[i])
documents # First 5 documents

[array([0.00347893, 0.00347827, 0.00347341, 0.0034797 , 0.6444762 ,
        0.33465561, 0.00347645, 0.00348142]),
 array([0.00266389, 0.00266446, 0.17470079, 0.00266435, 0.00266329,
        0.27134833, 0.54063089, 0.002664  ]),
 array([0.00347481, 0.00347479, 0.0034833 , 0.72049644, 0.00347599,
        0.00347686, 0.00347511, 0.2586427 ]),
 array([0.00833857, 0.22113744, 0.00833844, 0.00835289, 0.00833864,
        0.00834766, 0.00834197, 0.7288044 ]),
 array([0.00595988, 0.8111296 , 0.00595714, 0.00596588, 0.15309792,
        0.0059623 , 0.00596503, 0.00596225])]

In [40]:
# Assigning topics for the 5 docs
for i in range(0,5):
    print("document " + str(i) + ": topic", np.argmax(documents[i])) # get the index of the max score

document 0: topic 4
document 1: topic 6
document 2: topic 3
document 3: topic 7
document 4: topic 1


In [41]:
# printing the text to analyze the correctness
pd.set_option('display.max_colwidth', 300) # allowing the df to display the text fully so i can read it
df_train.head(5)['text_pp']

0    know read sence wonder help provide paragraph shall unlawful person transfer possess machinegun get look law dictionary find person artificial entity create government right federal constitution understand statute like enforce individual tell government tell possess pass law law know law constit...
1    bad question ref list algorithm think bit hard point find plane define circle lie algorithm find center circle line pass center perpendicular plane point pass center sphere repeat unused point original point give different line pass sphere origin interection center sphere radius easy compute dis...
2    Actuallay hand support idea have newsgroup aspect graphic programming propose Brian reply original posting suggest loose structure like reason make group possible reason spilt posting day take time browse posting find interested understand agree want aspect graphic meeting agree extension news f...
3                                                                                    

1. Document 0 was assigned topic 4, seems to talk about passing a law for machine guns or gun (our suggested label was matching)
2. Doc 1 was assigned topic 6, Seems to talk about algorithms and shapes, processing algorithms (matches our suggested label of processing)
3. Doc 2 was assigned topic 3, It seems that the model got this wrong, and most likely due to luck as looking at the scores, it seems the topic 7 is also high which i think i sthe correct one so this is probably the model not being perfect which is expected
4. Doc 3 was assigned topic 7, seems to talk about windows OS with stuff like root, windows, keypress, event, buttonPress, subwindow (topic 7 matches label)
5. Doc 4 was assigned topic 1, which seems to talk about the armenian genocide and the suggested label was matching

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

_Points:_ 6

1. In lecture 18, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 18 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

2. Transfer learning in NLP is the fine tuining of pre trained language models. We train these already trained models on our data as we did above and can use them to answer specific questions or automate things like topics of newspapers
   Transfer Learning in Computer vision works the same but it used more for things like object detection and image classifications.
3. Neural Networks benefit us with being able to extract generic features and learn them, as well as also specializing in later layers. They also perform better due to the pretrained layers which also allow for fine tuning.

<!-- END QUESTION -->

<br><br><br><br>

## Submission instructions 

**PLEASE READ:** When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using PrairieLearn.
4. Make sure that the plots and output are rendered properly in your submitted file.